# CS-513 Binary Classification Project
___
## Naive-Bayes and Logistic Regression on Airline Data

Wyatt Blair

In [17]:
import pandas as pd
import numpy as np
from sklearn.base import BaseEstimator
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import GaussianNB, CategoricalNB, BernoulliNB, MultinomialNB
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.model_selection import GridSearchCV
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import ConfusionMatrixDisplay, RocCurveDisplay, classification_report
import os
import json

import warnings
warnings.filterwarnings("ignore")

from src.util import test_classifier

In [18]:
# make output dir where charts will be stored
output_dir = './wyatt_output'
os.makedirs(output_dir, exist_ok=True)

In [19]:
data_2022_nans = pd.read_csv('./data/heart disease/2022/heart_2022_with_nans.csv')
data_2022_no_nans = pd.read_csv('./data/heart disease/2022/heart_2022_no_nans.csv')
data_2020_cleaned = pd.read_csv('./data/heart disease/2020/heart_2020_cleaned.csv')

## Data Pre-Processing

In [20]:
data = data_2022_nans

N, M = data.shape

In [21]:
target = 'HadHeartAttack'
features = data.columns.to_list(); features.remove(target)

In [22]:
data.head(5)

,State,Sex,GeneralHealth,PhysicalHealthDays,MentalHealthDays,LastCheckupTime,PhysicalActivities,SleepHours,RemovedTeeth,HadHeartAttack,...,HeightInMeters,WeightInKilograms,BMI,AlcoholDrinkers,HIVTesting,FluVaxLast12,PneumoVaxEver,TetanusLast10Tdap,HighRiskLastYear,CovidPos
0,Alabama,Female,Very good,0.0,0.0,Within past year (anytime less than 12 months ...,No,8.0,NaN,No,...,NaN,NaN,NaN,No,No,Yes,No,"Yes, received tetanus shot but not sure what type",No,No
1,Alabama,Female,Excellent,0.0,0.0,NaN,No,6.0,NaN,No,...,1.60,68.04,26.57,No,No,No,No,"No, did not receive any tetanus shot in the pa...",No,No
2,Alabama,Female,Very good,2.0,3.0,Within past year (anytime less than 12 months ...,Yes,5.0,NaN,No,...,1.57,63.50,25.61,No,No,No,No,NaN,No,Yes
3,Alabama,Female,Excellent,0.0,0.0,Within past year (anytime less than 12 months ...,Yes,7.0,NaN,No,...,1.65,63.50,23.30,No,No,Yes,Yes,"No, did not receive any tetanus shot in the pa...",No,No
4,Alabama,Female,Fair,2.0,0.0,Within past year (anytime less than 12 months ...,Yes,9.0,NaN,No,...,1.57,53.98,21.77,Yes,No,No,Yes,"No, did not receive any tetanus shot in the pa...",No,No


In [23]:
def clean(X):

    clean_data = X.copy()

    # Drop NaN rows
    clean_data.dropna(inplace=True)

    # Label encoding for categorical columns
    num_bins = 3
    for col in clean_data:

        series = clean_data[col]
        dtype = series.dtype

        if dtype in [np.float_, np.int_]: 

            clean_data[col] = pd.cut(series, bins=num_bins, labels=False)

        else:

            le = LabelEncoder()
            encoded_series = le.fit_transform(series.values)

            clean_data[col] = encoded_series

    return clean_data


In [24]:
clean_data = clean(data)

___

In [25]:
split_frac = 0.6

train = clean_data.sample(frac=split_frac)
test = clean_data.drop(index=train.index)

In [26]:
# undersample to remove bias towards '0' (a.k.a. no heart attack) target value
no_yes_ratio = 1
value_counts = train[target].value_counts()

no_heart_attack_data  = train[clean_data[target] == 0].sample(int(np.ceil(value_counts[1] * no_yes_ratio)))
yes_heart_attack_data = train[clean_data[target] == 1]

undersampled_train = pd.concat([no_heart_attack_data, yes_heart_attack_data])
undersampled_train[target].value_counts()

HadHeartAttack
0    8038
1    8038
Name: count, dtype: int64

In [27]:
X_train = train[features]
y_train = train[target]

X_test = test[features]
y_test = test[target]

# =================================================

X_undersampled_train = undersampled_train[features]
y_undersampled_train = undersampled_train[target]

In [28]:
neg_scaler = MinMaxScaler().fit(X_train)

X_train = neg_scaler.transform(X_train)
X_test = neg_scaler.transform(X_test)

X_undersampled_train = neg_scaler.transform(X_undersampled_train)

In [29]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(147613, 39)
(147613,)
(98409, 39)
(98409,)


___
## Find optimal models

In [30]:
def get_optimal_models(scoring):

    # Set parameters
    cat_nb_params = {
        'alpha': np.logspace(0, 100, num=5),
        'force_alpha': [True, False],
        'fit_prior': [True, False],
        'min_categories': [None] + list(np.arange(1, len(features), step=10)),
    }

    bern_nb_params = {
        'alpha': np.logspace(0, 100, num=50),
        'force_alpha': [True, False],
        'fit_prior': [True, False],

    }

    gauss_nb_params = {
        'var_smoothing': np.logspace(1e-12, 1e3, num=200),

    }

    log_reg_params = {
        'penalty' : ['l1', 'l2', 'elasticnet'],
        'C' : np.logspace(-4, 4, num=5),
        'solver' : [
            'liblinear', 
            'newton-cg', 
            'newton-cholesky', 
            'sag', 
            'saga'
        ],
        'max_iter': [int(N/5)],
        'random_state': [42],

    }

    # ========================================

    # Define GridSearch Objects
    log_reg = GridSearchCV(
        LogisticRegression(),
        param_grid=log_reg_params, 
        cv=5, 
        verbose=1, 
        n_jobs=-1,
        scoring=scoring
    )

    cat_nb = GridSearchCV(
        CategoricalNB(),
        param_grid=cat_nb_params, 
        cv=5, 
        verbose=1, 
        n_jobs=-1,
        scoring=scoring
    )

    bern_nb = GridSearchCV(
        BernoulliNB(),
        param_grid=bern_nb_params, 
        cv=5, 
        verbose=1, 
        n_jobs=-1,
        scoring=scoring
    )

    gauss_nb = GridSearchCV(
        GaussianNB(),
        param_grid=gauss_nb_params, 
        cv=5, 
        verbose=1, 
        n_jobs=-1,
        scoring=scoring
    )

    # Fit Grid Search Objects
    log_reg.fit(X_train, y_train)
    cat_nb.fit(X_train, y_train)
    bern_nb.fit(X_train, y_train)
    gauss_nb.fit(X_train, y_train)

    # =================================================

    # Undersampled data
    undersampled_log_reg = GridSearchCV(
        LogisticRegression(),
        param_grid=log_reg_params, 
        cv=5, 
        verbose=1, 
        n_jobs=-1,
        scoring=scoring
    )

    undersampled_cat_nb = GridSearchCV(
        CategoricalNB(),
        param_grid=cat_nb_params, 
        cv=5, 
        verbose=1, 
        n_jobs=-1,
        scoring=scoring
    )

    undersampled_bern_nb = GridSearchCV(
        BernoulliNB(),
        param_grid=bern_nb_params, 
        cv=5, 
        verbose=1, 
        n_jobs=-1,
        scoring=scoring
    )

    undersampled_gauss_nb = GridSearchCV(
        GaussianNB(),
        param_grid=gauss_nb_params, 
        cv=5, 
        verbose=1, 
        n_jobs=-1,
        scoring=scoring
    )

    undersampled_log_reg.fit(X_undersampled_train, y_undersampled_train)
    undersampled_cat_nb.fit(X_undersampled_train, y_undersampled_train)
    undersampled_bern_nb.fit(X_undersampled_train, y_undersampled_train)
    undersampled_gauss_nb.fit(X_undersampled_train, y_undersampled_train)

    regular = (log_reg, cat_nb, bern_nb, gauss_nb)
    undersampled = (undersampled_log_reg, undersampled_cat_nb, undersampled_bern_nb, undersampled_gauss_nb)

    return regular, undersampled


In [31]:
def evaluate_model(model: GridSearchCV, full_name: str, X_test: pd.DataFrame, y_test: pd.Series):

    scoring = full_name.split('-')[0]
    name = full_name.split('-')[1]

    # define filepaths
    model_output_fp = os.path.join(output_dir, scoring, name)
    os.makedirs(model_output_fp, exist_ok=True)

    confusion_matrix_fp = os.path.join(model_output_fp, 'confusion_matrix.png')
    roc_fp = os.path.join(model_output_fp, 'roc.png')
    param_dict_fp = os.path.join(model_output_fp, 'params.json')
    report_fp = os.path.join(model_output_fp, 'classification_report.txt')

    # have model make prediction on test set
    y_pred = model.predict(X_test)

    # save confusion matrix
    fig, confusion_matrix_ax = plt.subplots(1,1, figsize=(10,10))
    _ = ConfusionMatrixDisplay.from_predictions(
        y_test, y_pred, ax=confusion_matrix_ax
    )
    confusion_matrix_ax.set_title(f"{full_name} Confusion Matrix")
    plt.savefig(confusion_matrix_fp)
    plt.clf()
    plt.close()

    # save roc curve
    fig, roc_ax = plt.subplots(1,1, figsize=(10,10))
    _ = RocCurveDisplay.from_estimator(
        model, X_test, y_test, ax=roc_ax
    )
    roc_ax.set_title(f"{full_name} ROC")
    plt.savefig(roc_fp)
    plt.clf()
    plt.close()

    # save param dict
    params = model.best_estimator_.get_params()
    with open(param_dict_fp, 'w') as f:
        json.dump(params, f, indent=4, default=lambda o: '<not serializable>')
    
    # save classification report
    report = classification_report(y_true=y_test, y_pred=y_pred)
    with open(report_fp, 'w') as f:
        f.writelines(report)

# perform GridSearchCV and take the best model depending on the desired scoring 
def train_models(scoring):

    regular, undersampled  = get_optimal_models(scoring=scoring)

    (undersampled_log_reg, undersampled_cat_nb, undersampled_bern_nb, undersampled_gauss_nb) = undersampled
    (log_reg, cat_nb, bern_nb, gauss_nb) = regular

    models = {
        
        'Undersampled Logistic Regression': undersampled_log_reg,
        'Undersampled Categorical Naive Bayes': undersampled_cat_nb,
        'Undersampled Bernoulli Naive Bayes': undersampled_bern_nb,
        'Undersampled Gaussian Naive Bayes': undersampled_gauss_nb,

        'Logistic Regression': log_reg,
        'Categorical Naive Bayes': cat_nb,
        'Bernoulli Naive Bayes': bern_nb,
        'Gaussian Naive Bayes': gauss_nb,
        
    }

    return models

def evaluate_models(scoring, models):

    for name, model in models.items():
        evaluate_model(model=model, full_name=f"{scoring}-{name}", X_test=X_test, y_test=y_test)

In [32]:
f1_models        = train_models('f1'       )
recall_models    = train_models('recall'   )
roc_auc_models   = train_models('roc_auc'  )
jaccard_models   = train_models('jaccard'  )
precision_models = train_models('precision')

Fitting 5 folds for each of 75 candidates, totalling 375 fits
Fitting 5 folds for each of 100 candidates, totalling 500 fits
Fitting 5 folds for each of 200 candidates, totalling 1000 fits
Fitting 5 folds for each of 200 candidates, totalling 1000 fits
Fitting 5 folds for each of 75 candidates, totalling 375 fits
Fitting 5 folds for each of 100 candidates, totalling 500 fits
Fitting 5 folds for each of 200 candidates, totalling 1000 fits
Fitting 5 folds for each of 200 candidates, totalling 1000 fits
Fitting 5 folds for each of 75 candidates, totalling 375 fits
Fitting 5 folds for each of 100 candidates, totalling 500 fits
Fitting 5 folds for each of 200 candidates, totalling 1000 fits
Fitting 5 folds for each of 200 candidates, totalling 1000 fits
Fitting 5 folds for each of 75 candidates, totalling 375 fits
Fitting 5 folds for each of 100 candidates, totalling 500 fits
Fitting 5 folds for each of 200 candidates, totalling 1000 fits
Fitting 5 folds for each of 200 candidates, totallin

/Users/wyatthenryblair/anaconda3/envs/torch20/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/wyatthenryblair/anaconda3/envs/torch20/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/wyatthenryblair/anaconda3/envs/torch20/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/wyatthenryblair

Fitting 5 folds for each of 100 candidates, totalling 500 fits


/Users/wyatthenryblair/anaconda3/envs/torch20/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/wyatthenryblair/anaconda3/envs/torch20/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/wyatthenryblair/anaconda3/envs/torch20/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/wyatthenryblair

Fitting 5 folds for each of 200 candidates, totalling 1000 fits


...

/Users/wyatthenryblair/anaconda3/envs/torch20/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/wyatthenryblair/anaconda3/envs/torch20/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/wyatthenryblair/anaconda3/envs/torch20/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/wyatthenryblair

Fitting 5 folds for each of 200 candidates, totalling 1000 fits


/Users/wyatthenryblair/anaconda3/envs/torch20/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/wyatthenryblair/anaconda3/envs/torch20/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/wyatthenryblair/anaconda3/envs/torch20/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/wyatthenryblair

Fitting 5 folds for each of 200 candidates, totalling 1000 fits


/Users/wyatthenryblair/anaconda3/envs/torch20/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/wyatthenryblair/anaconda3/envs/torch20/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/wyatthenryblair/anaconda3/envs/torch20/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/wyatthenryblair

In [33]:
evaluate_models(scoring='precision', models=precision_models)

In [ ]:
evaluate_models(scoring='recall', models=recall_models)

In [ ]:
evaluate_models(scoring='f1', models=f1_models)

In [ ]:
evaluate_models(scoring='roc_auc', models=roc_auc_models)

In [ ]:
evaluate_models(scoring='jaccard', models=jaccard_models)